# Data Preparation 2

This notebook consist in the second data preparation. Instances of admission Height and Weight measures available corresponding to each ICUSTAY_ID from CHARTEVENTS.db database created in the first notebook (DataPreparation_1) are taken and converted to pandas Dataframe in order to fill the desired features in the main file MAIN_DF.csv. This features are selected via ITEMID. Then Body Mass Index (BMI) is computed from this two features.

This example is the ilustration of the process that will be more automatic in the following notebooks. The goal is to create a rich Dataframe of features for each given admission to the ICU with ICUSTAY_ID identification.

In [325]:
import pandas as pd
import numpy as np
import sqlite3
from IPython.display import display
pd.options.display.max_columns = None

Take the file created in DataPreparation_1

In [326]:
admissions=pd.read_csv('MAIN_DF.csv')
admissions=admissions.drop(['Unnamed: 0'], axis=1)
admissions=admissions.reset_index(drop=True)
print(admissions.shape)
print('unique SUBJECT_ID:', admissions.SUBJECT_ID.nunique())
print('unique HADM_ID   :', admissions.HADM_ID.nunique())
print('unique ICUSTAY_ID:', admissions.ICUSTAY_ID.nunique())
admissions.head()

(48989, 25)
unique SUBJECT_ID: 36659
unique HADM_ID   : 46273
unique ICUSTAY_ID: 48989


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,GENDER,AGE_AD,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,DIAGNOSIS,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,LOS_C,TIMEDELTA,TIMEDELTA_C,HOSPITAL_EXPIRE_FLAG
0,3,145834,211552,M,76.53,2101-10-20 19:08:00,2101-10-31 13:58:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,WHITE,HYPOTENSION,carevue,MICU,MICU,12,12,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646,>4,10.78,>10,0
1,4,185777,294638,F,47.84,2191-03-16 00:28:00,2191-03-23 18:41:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,WHITE,"FEVER,DEHYDRATION,FAILURE TO THRIVE",carevue,MICU,MICU,52,52,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785,1-2,7.76,6-10,0
2,6,107064,228232,F,65.94,2175-05-30 07:15:00,2175-06-15 16:00:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,WHITE,CHRONIC RENAL FAILURE/SDA,carevue,SICU,SICU,33,33,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729,2-4,16.36,>10,0
3,9,150750,220597,M,41.79,2149-11-09 13:06:00,2149-11-14 10:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicaid,NW,HEMORRHAGIC CVA,carevue,MICU,MICU,15,15,2149-11-09 13:07:02,2149-11-14 20:52:14,5.3231,>4,4.88,3-6,1
4,11,194540,229441,F,50.15,2178-04-16 06:18:00,2178-05-11 19:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,WHITE,BRAIN MASS,carevue,SICU,SICU,57,57,2178-04-16 06:19:32,2178-04-17 20:21:05,1.5844,1-2,25.53,>10,0


Define the connexion to the database where we are going to take the desired instances

In [327]:
connex = sqlite3.connect("data/CHARTEVENTS.db")
cur = connex.cursor()

Height in centimeters extraction.

From D_ITEMS.csv file we can link each ITEMID to a measure of a concrete variable or feature

For the Height in centimeters its ITEMID is 226730

In [328]:
ids = [226730]
ids = [str(id) for id in ids] 
str_matching = "(" + ",".join(ids) + ")"
print(str_matching)

(226730)


In [329]:
sql = "SELECT * FROM CHARTEVENTS_DB WHERE ITEMID IN " + str_matching + ";"
print('String of SQL   :', sql)
print('Object Execution:', cur.execute(sql))

String of SQL   : SELECT * FROM CHARTEVENTS_DB WHERE ITEMID IN (226730);
Object Execution: <sqlite3.Cursor object at 0x7fba246cfdc0>


In [330]:
df_226730 = pd.read_sql_query(sql, connex)
print(df_226730.shape)
df_226730.head()

(12015, 15)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
0,443,34,144319,290505.0,226730,2191-02-23 11:25:00,2191-02-23 11:25:00,15173.0,165.0,165.0,cm,0,0,None,None
1,1981,36,165660,241249.0,226730,2134-05-16 10:58:00,2134-05-16 10:58:00,16223.0,180.0,180.0,cm,0,0,None,None
2,3010,107,174162,264253.0,226730,2122-05-15 23:40:00,2122-05-15 23:40:00,17114.0,168.0,168.0,cm,0,0,None,None
3,3406,109,147469,253139.0,226730,2141-06-11 21:27:00,2141-06-11 21:27:00,17248.0,152.0,152.0,cm,0,0,None,None
4,4328,109,131345,243978.0,226730,2141-09-05 22:15:00,2141-09-05 22:15:00,19937.0,150.0,150.0,cm,0,0,None,None


In [331]:
print('unique ICUSTAY_ID:', df_226730.ICUSTAY_ID.nunique())

unique ICUSTAY_ID: 12011


In [332]:
df_226730.isnull().sum()

ROW_ID              0
SUBJECT_ID          0
HADM_ID             0
ICUSTAY_ID          3
ITEMID              0
CHARTTIME           0
STORETIME           0
CGID                0
VALUE               0
VALUENUM            0
VALUEUOM            0
WARNING             0
ERROR               0
RESULTSTATUS    12015
STOPPED         12015
dtype: int64

In [333]:
df_226730=df_226730[df_226730['ERROR']==0]
df_226730=df_226730.drop(['ROW_ID', 'ITEMID', 'STORETIME', 'CGID', 'VALUENUM', 'VALUEUOM', 'WARNING', 
                                'ERROR', 'RESULTSTATUS', 'STOPPED'], axis=1)

In [334]:
df_226730=df_226730.rename({'CHARTTIME':'T_HEIGHT', 'VALUE':'HEIGHT'}, axis='columns')
print(df_226730.shape)
df_226730.head()

(12015, 5)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,T_HEIGHT,HEIGHT
0,34,144319,290505.0,2191-02-23 11:25:00,165.0
1,36,165660,241249.0,2134-05-16 10:58:00,180.0
2,107,174162,264253.0,2122-05-15 23:40:00,168.0
3,109,147469,253139.0,2141-06-11 21:27:00,152.0
4,109,131345,243978.0,2141-09-05 22:15:00,150.0


Merge with the original file conserving the keys of the original file. This is because the second new file doesn't contain values for some ICUSTAY_ID

In [335]:
admissions=pd.merge(admissions,df_226730,how='left',on=['SUBJECT_ID', 'HADM_ID','ICUSTAY_ID'])
print(admissions.shape)
admissions=admissions.sort_values(by=['T_HEIGHT'])
admissions=admissions.reset_index(drop=True)
admissions.head()

(48989, 27)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,GENDER,AGE_AD,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,DIAGNOSIS,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,LOS_C,TIMEDELTA,TIMEDELTA_C,HOSPITAL_EXPIRE_FLAG,T_HEIGHT,HEIGHT
0,41552,120254,268176,M,72.110000,2100-07-05 13:52:00,2100-07-08 15:45:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NW,ANEMIA,metavision,MICU,MICU,52,52,2100-07-06 13:59:09,2100-07-07 15:11:01,1.0499,1-2,3.08,3-6,0,2100-07-06 16:04:00,157.0
1,31288,136246,250905,F,93.369611,2100-07-16 11:29:00,2100-07-22 18:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,WHITE,INTRACRANIAL HEMORRHAGE,metavision,TSICU,TSICU,14,14,2100-07-16 11:30:04,2100-07-17 16:31:25,1.2093,1-2,6.27,6-10,0,2100-07-16 17:15:00,160.0
2,71582,101422,218983,M,76.470000,2100-07-16 18:09:00,2100-07-24 20:07:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,NW,AORTIC STENOSIS,metavision,CSRU,CSRU,12,12,2100-07-20 09:07:25,2100-07-21 11:03:26,1.0806,1-2,8.08,6-10,0,2100-07-20 11:21:00,160.0
3,31263,198779,278147,M,18.540000,2100-07-18 23:51:00,2100-07-27 12:54:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,NW,GUNSHOT WOUND TO FACE,both,TSICU,TSICU,14,14,2100-07-18 23:52:23,2100-07-23 16:49:17,4.7062,>4,8.54,6-10,0,2100-07-20 16:43:00,168.0
4,51754,153020,235889,M,45.050000,2100-07-27 18:12:00,2100-07-31 01:25:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,WHITE,GASTROINTESTINAL BLEED,metavision,MICU,MICU,23,23,2100-07-27 18:13:04,2100-07-31 13:25:27,3.8003,2-4,3.30,3-6,0,2100-07-27 18:37:00,188.0


In [336]:
admissions=admissions.drop_duplicates('ICUSTAY_ID')

In [337]:
admissions.shape

(48989, 27)

In [338]:
admissions.isnull().sum()

SUBJECT_ID                  0
HADM_ID                     0
ICUSTAY_ID                  0
GENDER                      0
AGE_AD                      0
ADMITTIME                   0
DISCHTIME                   0
ADMISSION_TYPE              0
ADMISSION_LOCATION          0
DISCHARGE_LOCATION          0
INSURANCE                   0
ETHNICITY                   0
DIAGNOSIS                   0
DBSOURCE                    0
FIRST_CAREUNIT              0
LAST_CAREUNIT               0
FIRST_WARDID                0
LAST_WARDID                 0
INTIME                      0
OUTTIME                     0
LOS                         0
LOS_C                       0
TIMEDELTA                   0
TIMEDELTA_C                 0
HOSPITAL_EXPIRE_FLAG        0
T_HEIGHT                37951
HEIGHT                  37951
dtype: int64

Weight admission in kilograms extraction

In [339]:
ids = [226512]
ids = [str(id) for id in ids] 
str_matching = "(" + ",".join(ids) + ")"
print(str_matching)

(226512)


In [340]:
sql = "SELECT * FROM CHARTEVENTS_DB WHERE ITEMID IN " + str_matching + ";"
print('String of SQL   :', sql)
print('Object Execution:', cur.execute(sql))

String of SQL   : SELECT * FROM CHARTEVENTS_DB WHERE ITEMID IN (226512);
Object Execution: <sqlite3.Cursor object at 0x7fba246cfdc0>


In [341]:
df_226512 = pd.read_sql_query(sql, connex)
print(df_226512.shape)
df_226512.head()

(22604, 15)


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
0,355,34,144319,290505.0,226512,2191-02-23 07:44:00,2191-02-23 07:44:00,17741.0,74.5,74.5,kg,0,0,None,None
1,8,23,124321,234044.0,226512,2157-10-21 12:15:00,2157-10-21 12:15:00,16978.0,66.8,66.8,kg,0,0,None,None
2,1978,36,165660,241249.0,226512,2134-05-16 10:58:00,2134-05-16 10:58:00,16223.0,106.2,106.2,kg,0,0,None,None
3,2029,85,112077,291697.0,226512,2167-07-25 21:31:00,2167-07-25 21:31:00,21050.0,98.0,98.0,kg,0,0,None,None
4,2505,107,182383,252542.0,226512,2121-12-01 05:54:00,2121-12-01 05:54:00,16526.0,88.6,88.6,kg,0,0,None,None


In [342]:
print('unique ICUSTAY_ID:', df_226512.ICUSTAY_ID.nunique())

unique ICUSTAY_ID: 22597


In [343]:
df_226512.isnull().sum()

ROW_ID              0
SUBJECT_ID          0
HADM_ID             0
ICUSTAY_ID          5
ITEMID              0
CHARTTIME           0
STORETIME           0
CGID                0
VALUE               0
VALUENUM            0
VALUEUOM            0
WARNING             0
ERROR               0
RESULTSTATUS    22604
STOPPED         22604
dtype: int64

In [344]:
df_226512=df_226512[df_226512['ERROR']==0]
df_226512=df_226512.drop(['ROW_ID', 'ITEMID', 'STORETIME', 'CGID', 'VALUENUM', 'VALUEUOM', 'WARNING', 
                                'ERROR', 'RESULTSTATUS', 'STOPPED'], axis=1)

In [345]:
df_226512=df_226512.rename({'CHARTTIME':'T_WEIGHT', 'VALUE':'WEIGHT'}, axis='columns')
print(df_226512.shape)
df_226512.head()

(22604, 5)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,T_WEIGHT,WEIGHT
0,34,144319,290505.0,2191-02-23 07:44:00,74.5
1,23,124321,234044.0,2157-10-21 12:15:00,66.8
2,36,165660,241249.0,2134-05-16 10:58:00,106.2
3,85,112077,291697.0,2167-07-25 21:31:00,98.0
4,107,182383,252542.0,2121-12-01 05:54:00,88.6


In [346]:
admissions=pd.merge(admissions,df_226512,how='left',on=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])
print(admissions.shape)
admissions=admissions.sort_values(by=['T_WEIGHT'])
admissions=admissions.reset_index(drop=True)
admissions.head()

(48990, 29)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,GENDER,AGE_AD,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,DIAGNOSIS,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,LOS_C,TIMEDELTA,TIMEDELTA_C,HOSPITAL_EXPIRE_FLAG,T_HEIGHT,HEIGHT,T_WEIGHT,WEIGHT
0,82574,118464,202134,M,56.120000,2100-06-07 19:59:00,2100-06-09 17:09:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicaid,NW,HYPOTENSION,metavision,MICU,MICU,23,52,2100-06-07 20:00:22,2100-06-08 14:59:31,0.7911,0-1,1.88,0-3,0,NaN,NaN,2100-06-08 00:21:00,74.5
1,31585,125380,234741,M,92.215338,2100-07-02 19:28:00,2100-07-07 18:05:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,WHITE,PNEUMONIA,metavision,MICU,MICU,50,50,2100-07-02 22:27:28,2100-07-04 01:57:28,1.1458,1-2,4.94,3-6,0,NaN,NaN,2100-07-03 01:49:00,87.0
2,79168,125272,293960,F,60.640000,2100-07-05 15:22:00,2100-07-27 17:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,REHAB/DISTINCT PART HOSP,Private,WHITE,MALAISE S/P SLEEVE GASTRECTOMY,metavision,SICU,SICU,33,33,2100-07-06 15:02:22,2100-07-15 13:14:41,8.9252,>4,22.07,>10,0,NaN,NaN,2100-07-06 15:10:00,126.8
3,41552,120254,268176,M,72.110000,2100-07-05 13:52:00,2100-07-08 15:45:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NW,ANEMIA,metavision,MICU,MICU,52,52,2100-07-06 13:59:09,2100-07-07 15:11:01,1.0499,1-2,3.08,3-6,0,2100-07-06 16:04:00,157.0,2100-07-06 16:04:00,74.9
4,72723,146616,267483,M,21.900000,2100-07-09 10:43:00,2100-07-17 15:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Private,WHITE,ABDOMINAL PAIN;JAUNDICE,metavision,SICU,SICU,33,33,2100-07-09 13:19:40,2100-07-10 16:19:58,1.1252,1-2,8.18,6-10,0,NaN,NaN,2100-07-09 16:20:00,79.0


In [347]:
admissions=admissions.drop_duplicates('ICUSTAY_ID')

In [348]:
admissions['BMI']=np.round((admissions['WEIGHT']/(admissions['HEIGHT']*admissions['HEIGHT']/10000)), 2)

In [349]:
print(admissions.shape)
admissions.head()

(48989, 30)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,GENDER,AGE_AD,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,DIAGNOSIS,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,LOS_C,TIMEDELTA,TIMEDELTA_C,HOSPITAL_EXPIRE_FLAG,T_HEIGHT,HEIGHT,T_WEIGHT,WEIGHT,BMI
0,82574,118464,202134,M,56.120000,2100-06-07 19:59:00,2100-06-09 17:09:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicaid,NW,HYPOTENSION,metavision,MICU,MICU,23,52,2100-06-07 20:00:22,2100-06-08 14:59:31,0.7911,0-1,1.88,0-3,0,NaN,NaN,2100-06-08 00:21:00,74.5,NaN
1,31585,125380,234741,M,92.215338,2100-07-02 19:28:00,2100-07-07 18:05:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,WHITE,PNEUMONIA,metavision,MICU,MICU,50,50,2100-07-02 22:27:28,2100-07-04 01:57:28,1.1458,1-2,4.94,3-6,0,NaN,NaN,2100-07-03 01:49:00,87.0,NaN
2,79168,125272,293960,F,60.640000,2100-07-05 15:22:00,2100-07-27 17:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,REHAB/DISTINCT PART HOSP,Private,WHITE,MALAISE S/P SLEEVE GASTRECTOMY,metavision,SICU,SICU,33,33,2100-07-06 15:02:22,2100-07-15 13:14:41,8.9252,>4,22.07,>10,0,NaN,NaN,2100-07-06 15:10:00,126.8,NaN
3,41552,120254,268176,M,72.110000,2100-07-05 13:52:00,2100-07-08 15:45:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,NW,ANEMIA,metavision,MICU,MICU,52,52,2100-07-06 13:59:09,2100-07-07 15:11:01,1.0499,1-2,3.08,3-6,0,2100-07-06 16:04:00,157.0,2100-07-06 16:04:00,74.9,30.39
4,72723,146616,267483,M,21.900000,2100-07-09 10:43:00,2100-07-17 15:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Private,WHITE,ABDOMINAL PAIN;JAUNDICE,metavision,SICU,SICU,33,33,2100-07-09 13:19:40,2100-07-10 16:19:58,1.1252,1-2,8.18,6-10,0,NaN,NaN,2100-07-09 16:20:00,79.0,NaN


In [350]:
admissions[admissions['SUBJECT_ID']==23].sort_values(by=['ICUSTAY_ID'])

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,GENDER,AGE_AD,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,DIAGNOSIS,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,LOS_C,TIMEDELTA,TIMEDELTA_C,HOSPITAL_EXPIRE_FLAG,T_HEIGHT,HEIGHT,T_WEIGHT,WEIGHT,BMI
21436,23,152223,227807,M,71.13,2153-09-03 07:15:00,2153-09-08 19:10:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,carevue,CSRU,CSRU,14,14,2153-09-03 09:38:55,2153-09-04 15:59:11,1.2641,1-2,5.50,3-6,0,NaN,NaN,NaN,NaN,NaN
11942,23,124321,234044,M,75.25,2157-10-18 19:34:00,2157-10-25 14:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,WHITE,BRAIN MASS,metavision,SICU,SICU,57,57,2157-10-21 11:40:38,2157-10-22 16:08:48,1.1862,1-2,6.77,6-10,0,NaN,NaN,2157-10-21 12:15:00,66.8,NaN


In [351]:
admissions[admissions['SUBJECT_ID']==109].sort_values(by=['ICUSTAY_ID']).head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,GENDER,AGE_AD,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,DIAGNOSIS,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,LOS_C,TIMEDELTA,TIMEDELTA_C,HOSPITAL_EXPIRE_FLAG,T_HEIGHT,HEIGHT,T_WEIGHT,WEIGHT,BMI
21505,109,183350,208379,F,20.25,2137-11-04 19:36:00,2137-11-21 18:13:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Government,NW,HYPERTENSIVE EMERGENCY,carevue,MICU,MICU,12,12,2137-11-14 20:47:58,2137-11-20 22:38:41,6.0769,>4,16.94,>10,0,NaN,NaN,NaN,NaN,NaN
8729,109,196721,214027,F,24.93,2142-07-12 15:27:00,2142-07-23 18:41:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,NW,SHORTNESS OF BREATH,metavision,CCU,CCU,7,7,2142-07-12 15:28:40,2142-07-15 20:45:04,3.2197,2-4,11.13,>10,0,NaN,NaN,2142-07-12 17:35:00,50.2,NaN
21513,109,196721,217934,F,24.93,2142-07-12 15:27:00,2142-07-23 18:41:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,NW,SHORTNESS OF BREATH,metavision,CCU,CCU,7,7,2142-07-17 07:48:43,2142-07-18 20:28:26,1.5276,1-2,11.13,>10,0,NaN,NaN,NaN,NaN,NaN
8683,109,151240,219154,F,24.77,2142-05-15 10:45:00,2142-05-18 15:58:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicaid,NW,HYPERTENSIVE CRISIS,metavision,TSICU,TSICU,14,14,2142-05-15 10:46:27,2142-05-16 10:44:24,0.9986,0-1,3.22,3-6,0,NaN,NaN,2142-05-15 12:29:00,42.0,NaN
21511,109,110668,221610,F,23.05,2140-08-25 14:39:00,2140-09-02 18:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicaid,NW,HYPERTENSION EMERGENCY,carevue,MICU,MICU,23,23,2140-08-25 14:40:48,2140-08-31 23:34:12,6.3704,>4,8.16,6-10,0,NaN,NaN,NaN,NaN,NaN


In [352]:
a_h=pd.to_datetime(admissions.T_HEIGHT)-pd.to_datetime(admissions.INTIME)
a_w=pd.to_datetime(admissions.T_WEIGHT)-pd.to_datetime(admissions.INTIME)
admissions.T_HEIGHT=np.round(a_h.dt.total_seconds()/86400, 2)
admissions.T_WEIGHT=np.round(a_w.dt.total_seconds()/86400, 2)

In [353]:
admissions[admissions['SUBJECT_ID']==23].sort_values(by=['ICUSTAY_ID'])

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,GENDER,AGE_AD,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,DIAGNOSIS,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,LOS_C,TIMEDELTA,TIMEDELTA_C,HOSPITAL_EXPIRE_FLAG,T_HEIGHT,HEIGHT,T_WEIGHT,WEIGHT,BMI
21436,23,152223,227807,M,71.13,2153-09-03 07:15:00,2153-09-08 19:10:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,carevue,CSRU,CSRU,14,14,2153-09-03 09:38:55,2153-09-04 15:59:11,1.2641,1-2,5.50,3-6,0,NaN,NaN,NaN,NaN,NaN
11942,23,124321,234044,M,75.25,2157-10-18 19:34:00,2157-10-25 14:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,WHITE,BRAIN MASS,metavision,SICU,SICU,57,57,2157-10-21 11:40:38,2157-10-22 16:08:48,1.1862,1-2,6.77,6-10,0,NaN,NaN,0.02,66.8,NaN


In [354]:
admissions[admissions['SUBJECT_ID']==109].sort_values(by=['ICUSTAY_ID']).head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,GENDER,AGE_AD,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,DIAGNOSIS,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,LOS_C,TIMEDELTA,TIMEDELTA_C,HOSPITAL_EXPIRE_FLAG,T_HEIGHT,HEIGHT,T_WEIGHT,WEIGHT,BMI
21505,109,183350,208379,F,20.25,2137-11-04 19:36:00,2137-11-21 18:13:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Government,NW,HYPERTENSIVE EMERGENCY,carevue,MICU,MICU,12,12,2137-11-14 20:47:58,2137-11-20 22:38:41,6.0769,>4,16.94,>10,0,NaN,NaN,NaN,NaN,NaN
8729,109,196721,214027,F,24.93,2142-07-12 15:27:00,2142-07-23 18:41:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,NW,SHORTNESS OF BREATH,metavision,CCU,CCU,7,7,2142-07-12 15:28:40,2142-07-15 20:45:04,3.2197,2-4,11.13,>10,0,NaN,NaN,0.09,50.2,NaN
21513,109,196721,217934,F,24.93,2142-07-12 15:27:00,2142-07-23 18:41:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,NW,SHORTNESS OF BREATH,metavision,CCU,CCU,7,7,2142-07-17 07:48:43,2142-07-18 20:28:26,1.5276,1-2,11.13,>10,0,NaN,NaN,NaN,NaN,NaN
8683,109,151240,219154,F,24.77,2142-05-15 10:45:00,2142-05-18 15:58:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicaid,NW,HYPERTENSIVE CRISIS,metavision,TSICU,TSICU,14,14,2142-05-15 10:46:27,2142-05-16 10:44:24,0.9986,0-1,3.22,3-6,0,NaN,NaN,0.07,42.0,NaN
21511,109,110668,221610,F,23.05,2140-08-25 14:39:00,2140-09-02 18:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicaid,NW,HYPERTENSION EMERGENCY,carevue,MICU,MICU,23,23,2140-08-25 14:40:48,2140-08-31 23:34:12,6.3704,>4,8.16,6-10,0,NaN,NaN,NaN,NaN,NaN


In [355]:
admissions=admissions.sort_values(by=['SUBJECT_ID'])
admissions=admissions.reset_index(drop=True)

In [356]:
admissions.isnull().sum()

SUBJECT_ID                  0
HADM_ID                     0
ICUSTAY_ID                  0
GENDER                      0
AGE_AD                      0
ADMITTIME                   0
DISCHTIME                   0
ADMISSION_TYPE              0
ADMISSION_LOCATION          0
DISCHARGE_LOCATION          0
INSURANCE                   0
ETHNICITY                   0
DIAGNOSIS                   0
DBSOURCE                    0
FIRST_CAREUNIT              0
LAST_CAREUNIT               0
FIRST_WARDID                0
LAST_WARDID                 0
INTIME                      0
OUTTIME                     0
LOS                         0
LOS_C                       0
TIMEDELTA                   0
TIMEDELTA_C                 0
HOSPITAL_EXPIRE_FLAG        0
T_HEIGHT                37951
HEIGHT                  37951
T_WEIGHT                27662
WEIGHT                  27662
BMI                     38044
dtype: int64

In order to fill some NaN we can group by SUBJECT_ID and replace NaN that a given SUBJECT_ID can have for different HADM_ID and/or ICUSTAY_ID with the mean value of the available values for a given feature. I think this is a reasonable assumption.

Group by SUBJECT_ID and replace NaN by mean values

In [357]:
admissions['WEIGHT'] = admissions['WEIGHT'].groupby(admissions['SUBJECT_ID']).transform(lambda x: x.fillna(x.mean()))
admissions['HEIGHT'] = admissions['HEIGHT'].groupby(admissions['SUBJECT_ID']).transform(lambda x: x.fillna(x.mean()))
admissions['BMI']    = np.round((admissions['WEIGHT']/(admissions['HEIGHT']*admissions['HEIGHT']/10000)), 2)
admissions['BMI']    = admissions['BMI'].groupby(admissions['SUBJECT_ID']).transform(lambda x: x.fillna(x.mean()))

In [358]:
admissions.isnull().sum()

SUBJECT_ID                  0
HADM_ID                     0
ICUSTAY_ID                  0
GENDER                      0
AGE_AD                      0
ADMITTIME                   0
DISCHTIME                   0
ADMISSION_TYPE              0
ADMISSION_LOCATION          0
DISCHARGE_LOCATION          0
INSURANCE                   0
ETHNICITY                   0
DIAGNOSIS                   0
DBSOURCE                    0
FIRST_CAREUNIT              0
LAST_CAREUNIT               0
FIRST_WARDID                0
LAST_WARDID                 0
INTIME                      0
OUTTIME                     0
LOS                         0
LOS_C                       0
TIMEDELTA                   0
TIMEDELTA_C                 0
HOSPITAL_EXPIRE_FLAG        0
T_HEIGHT                37951
HEIGHT                  33397
T_WEIGHT                27662
WEIGHT                  24260
BMI                     33398
dtype: int64

As we can see, we have replaced some NaN by reasonable numbers

In [359]:
admissions[admissions['SUBJECT_ID']==23].head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,GENDER,AGE_AD,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,DIAGNOSIS,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,LOS_C,TIMEDELTA,TIMEDELTA_C,HOSPITAL_EXPIRE_FLAG,T_HEIGHT,HEIGHT,T_WEIGHT,WEIGHT,BMI
15,23,124321,234044,M,75.25,2157-10-18 19:34:00,2157-10-25 14:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,WHITE,BRAIN MASS,metavision,SICU,SICU,57,57,2157-10-21 11:40:38,2157-10-22 16:08:48,1.1862,1-2,6.77,6-10,0,NaN,NaN,0.02,66.8,NaN
16,23,152223,227807,M,71.13,2153-09-03 07:15:00,2153-09-08 19:10:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,carevue,CSRU,CSRU,14,14,2153-09-03 09:38:55,2153-09-04 15:59:11,1.2641,1-2,5.50,3-6,0,NaN,NaN,NaN,66.8,NaN


In [360]:
admissions[admissions['SUBJECT_ID']==109].head().sort_values(by=['ICUSTAY_ID']).head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,GENDER,AGE_AD,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,ETHNICITY,DIAGNOSIS,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,LOS_C,TIMEDELTA,TIMEDELTA_C,HOSPITAL_EXPIRE_FLAG,T_HEIGHT,HEIGHT,T_WEIGHT,WEIGHT,BMI
89,109,102024,232807,F,24.78,2142-05-20 12:59:00,2142-06-05 16:18:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicaid,NW,ABDOMINAL PAIN,metavision,MICU,MICU,23,23,2142-06-01 18:50:29,2142-06-03 14:41:49,1.8273,1-2,16.14,>10,0,0.4,152.0,0.17,45.000000,19.48
90,109,102024,240251,F,24.78,2142-05-20 12:59:00,2142-06-05 16:18:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicaid,NW,ABDOMINAL PAIN,metavision,MICU,MICU,50,50,2142-05-20 17:40:15,2142-05-22 22:03:12,2.1826,2-4,16.14,>10,0,NaN,150.8,0.11,42.000000,18.47
87,109,183350,277062,F,20.25,2137-11-04 19:36:00,2137-11-21 18:13:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Government,NW,HYPERTENSIVE EMERGENCY,carevue,MICU,MICU,12,12,2137-11-04 19:37:58,2137-11-05 17:04:39,0.8935,0-1,16.94,>10,0,NaN,150.8,NaN,47.484615,20.88
88,109,136572,286860,F,24.86,2142-06-18 03:11:00,2142-06-20 16:34:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicaid,NW,ANEMIA,metavision,MICU,MICU,23,23,2142-06-18 03:13:11,2142-06-19 00:02:20,0.8675,0-1,2.56,0-3,0,NaN,150.8,0.08,48.600000,21.37
91,109,183350,298630,F,20.25,2137-11-04 19:36:00,2137-11-21 18:13:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Government,NW,HYPERTENSIVE EMERGENCY,carevue,MICU,MICU,15,15,2137-11-09 13:31:16,2137-11-12 17:56:27,3.1842,2-4,16.94,>10,0,NaN,150.8,NaN,47.484615,20.88


In [361]:
admissions.to_csv('MAIN_DF_2.csv')